In [29]:
# immport dependencies
from bs4 import BeautifulSoup
import pandas as pd
import requests
import pymongo
from pymongo import MongoClient
import geocoder

In [30]:
# MongoDB connection
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [31]:
# creates repeater db if it doesn't already exist
# drops stations table to avoid duplicate entries
db = client.repeater_db
db.stations.drop()

In [32]:
# url to scrape & soup setup
url = 'https://www.repeaterbook.com/repeaters/Display_SS.php?state_id=06&band=4&loc=%&call=%&use=%'

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [33]:
# initialize variable to concat to location for use w/geocoder
california = ', CA'

In [ ]:
# scrapes url & pushes data to stations table in db
table_rows = soup.find_all('tr')[4:1000]

# for loop to pull out data from each result
for item in table_rows:
    freq = item.find('a').text
    call_sign = item.find_all('td', attrs={'class': None})[3].text
    county = item.find_all('td', attrs={'class': None})[2].text
    location = item.find(class_="w3-left-align").text
    usage = item.find('font').text.strip()
    
    # pulls location coordinates from geocoder
    lat = geocoder.osm(location + california).lat
    lng = geocoder.osm(location + california).lng
    
    # push results to mongodb
    db.stations.insert_one(
        {'location': location,
         'latitude': lat,
         'longitude': lng,
         'frequency': freq,
         'call sign': call_sign,
         'county': county,
         'usage': usage})
                    
    print(location, lat, lng, freq, call_sign, county, usage)

Moreno Valley 33.937517 -117.2305944 425.4250 KC7NP Riverside OPEN
San Jose 37.3361905 -121.8905833 430.3000 W6IOS Santa Clara OPEN
Chico 39.7284945 -121.8374777 440.0000 KI6PNB Butte OPEN
Fremont 37.5482697 -121.9885719 440.0000 WA6FSP Alameda OPEN
Fresno 36.7295295 -119.708861260756 440.0025 N6VRC Fresno OPEN
Sonoma 38.5110803 -122.8473388 440.0125 W6PE Sonoma OPEN
Fremont 37.5482697 -121.9885719 440.0250 KI6KGN Alameda OPEN
Pacifica 37.6138253 -122.4869194 440.0250 N6TZE San Mateo OPEN
Ukiah 39.1501662 -123.2077861 440.0250 WA6RQZ Mendocino OPEN
Vacaville 38.3565773 -121.9877444 440.0250 WV6F Solano OPEN
Morgan Hill, Loma Prieta None None 440.0375 W6YYY Santa Clara OPEN
Hayward 37.6688205 -122.0807964 440.0500 KQ6YG Alameda OPEN
Napa, Mt St Helena 38.4984875 -122.461163 440.0500 W6CO Sonoma OPEN
Redding 40.5863563 -122.3916754 440.0500 NA0SA Shasta CLOSED
Clovis 36.8252277 -119.7029194 440.0625 WX6D Fresno OPEN
Fresno 36.7295295 -119.708861260756 440.0625 KE6YJC Fresno OPEN
Angwin 3

Oroville 39.5137752 -121.556359 440.9000 W6SCR Butte OPEN
Berkeley 37.8708393 -122.2728639 440.9250 KH8AF Alameda CLOSED
Novato 38.1061979 -122.5681191 440.9250 KH8AF Marin CLOSED
San Rafael 37.9735346 -122.5310874 440.9250 KH8AF Marin CLOSED
Santa Cruz 36.9749416 -122.0285259 440.9250 K6BJ Santa Cruz OPEN
Tahoe City 39.1689372 -120.1434787 440.9250 KH8AF Placer CLOSED
Vacaville 38.3565773 -121.9877444 440.9250 KH8AF Solano CLOSED
Gold Run 39.180731 -120.8557726 440.9500 WB6OHV Placer OPEN
Hayward 37.6688205 -122.0807964 440.9500 KB6LED Alameda OPEN
Redding 40.5863563 -122.3916754 440.9500 KB6HWZ Shasta OPEN
Sebastopol 38.3845125 -122.833250158374 440.9500 WA6FUL Sonoma OPEN
Columbia 32.7178183 -117.1673622 440.9750 N6EUO Tuolumne OPEN
Mineral 40.3469311 -121.5993148 440.9750 N6TZG Tehama OPEN
Portola Valley 37.3841067 -122.2352443 440.9750 KC6ULT San Mateo OPEN
Richmond 37.9357576 -122.3477486 440.9750 K6LOU Contra Costa CLOSED
Cameron Park 38.6781746 -120.99426 441.0000 WA6NHC El Dor

Napa, Mt St Helena 38.4984875 -122.461163 441.9000 W6CO Sonoma OPEN
Aptos 36.9787725 -121.8976335 441.9250 WA6HHQ Santa Cruz OPEN
Twain Harte 38.0337345 -120.238494078094 441.9250 W6YOP Tuolumne OPEN
Vacaville 38.3565773 -121.9877444 441.9250 W6YOP Solano CLOSED
Sacramento 38.5815719 -121.4943996 441.9500 K6YC Sacramento OPEN
San Jose 37.3361905 -121.8905833 441.9500 N6MPX Santa Clara OPEN
Saratoga, Saratoga Heights 38.7239855 -90.6102651 441.9500 KK6USZ Santa Clara OPEN
Orinda 37.8771476 -122.1796888 441.9750 W6CBS Contra Costa OPEN
Springville, Blue Ridge None None 441.9750 KE6WDX Tulare OPEN
Vacaville 38.3565773 -121.9877444 441.9750 W6RCA Solano OPEN
Eureka, Horse Mountain None None 442.0000 AE6R Humboldt OPEN
Palo Alto 37.4443293 -122.1598465 442.0000 WW6HP Santa Clara OPEN
Rocklin 38.7907339 -121.2357828 442.0000 N6UG Placer OPEN
Lemoore 36.3006495 -119.7827122 442.0250 N6BEN Kings OPEN
Milpitas 37.4282724 -121.9066238 442.0250 W6KCS Santa Clara OPEN
Salinas 36.6744117 -121.65503

Fairfield 38.2493581 -122.0399663 442.7750 KC6UJM Solano OPEN
Prunedale 36.7851358 -121.6666416 442.7750 W6DXW Monterey OPEN
Quincy 39.9309945 -120.953573947289 442.7750 KF6CCP Plumas OPEN
San Jose 37.3361905 -121.8905833 442.7750 KU6V Santa Clara OPEN
San Leandro 37.7249296 -122.1560768 442.7750 KB6NCL Alameda CLOSED
Palo Alto 37.4443293 -122.1598465 442.8000 K6OTR Santa Clara OPEN
Sacramento 38.5815719 -121.4943996 442.8000 W6AK Sacramento OPEN
Clearlake 38.9582307 -122.6263728 442.8250 WR6COP Lake CLOSED
San Jose 37.3361905 -121.8905833 442.8250 WR6COP Santa Clara CLOSED
South Lake Tahoe, Heavenly Valley Ski area California base facility None None 442.8250 W6SUV El Dorado OPEN
Redding 40.5863563 -122.3916754 442.8500 WR6TV Shasta CLOSED
San Jose 37.3361905 -121.8905833 442.8500 WA6MIA Santa Clara OPEN
Vacaville 38.3565773 -121.9877444 442.8500 AB6CQ Solano OPEN
Cobb 38.8221258 -122.7230451 442.8750 KE6QDW Lake OPEN
Foresthill 39.0019935 -120.825401814224 442.8750 KA6EBR Placer OPEN


Pioneer 38.4318551 -120.5718719 443.6250 K6MSR Amador CLOSED
San Jose 37.3361905 -121.8905833 443.6250 K6MSR Santa Clara CLOSED
Westley 37.549376 -121.1991036 443.6250 K6MSR Stanislaus CLOSED
Fresno 36.7295295 -119.708861260756 443.6500 KE6SHK Fresno OPEN
Grass Valley 39.2086655 -121.053123077412 443.6500 KG6BAJ Nevada OPEN
Monterey 36.600256 -121.8946388 443.6500 N6AMO Monterey OPEN
Pleasanton 37.6624312 -121.8746789 443.6500 K6TEA Alameda CLOSED
Los Altos 37.3790629 -122.116578 443.6750 K6MSR Santa Clara CLOSED
San Francisco 37.7792808 -122.4192363 443.6750 K6MSR San Francisco CLOSED
Corcoran, Castle Peak None None 443.7000 AA6GZ Fresno OPEN
Fremont 37.5482697 -121.9885719 443.7000 K6JJC Alameda OPEN
Walnut Grove 38.2503075 -121.504773626118 443.7000 WA6JIV Sacramento CLOSED
Coalinga, Joaquin Ridge 36.1346222 -120.3702869 443.7250 N6LYE Fresno OPEN
Coalinga, Joaquin Ridge 36.1346222 -120.3702869 443.7250 K6NOX Fresno OPEN
Fremont, Mt Allison None None 443.7250 N6HWI Alameda OPEN
Los 

...

Status code 429 from https://nominatim.openstreetmap.org/search: ERROR - 429 Client Error: Too Many Requests for url: https://nominatim.openstreetmap.org/search?q=Ramona%2C+CA&format=jsonv2&addressdetails=1&limit=1
Status code 429 from https://nominatim.openstreetmap.org/search: ERROR - 429 Client Error: Too Many Requests for url: https://nominatim.openstreetmap.org/search?q=Ramona%2C+CA&format=jsonv2&addressdetails=1&limit=1


Ramona None None 445.7600 KD6RSQ San Diego OPEN


Status code 429 from https://nominatim.openstreetmap.org/search: ERROR - 429 Client Error: Too Many Requests for url: https://nominatim.openstreetmap.org/search?q=Crestline%2C+CA&format=jsonv2&addressdetails=1&limit=1
Status code 429 from https://nominatim.openstreetmap.org/search: ERROR - 429 Client Error: Too Many Requests for url: https://nominatim.openstreetmap.org/search?q=Crestline%2C+CA&format=jsonv2&addressdetails=1&limit=1


Crestline None None 445.7800 K6PIN San Bernardino CLOSED


Status code 429 from https://nominatim.openstreetmap.org/search: ERROR - 429 Client Error: Too Many Requests for url: https://nominatim.openstreetmap.org/search?q=Altadena%2C+Mt.+Disappointment%2C+CA&format=jsonv2&addressdetails=1&limit=1


In [ ]:
# pushes mongo db table to pandas dataframe
stations = db.stations
df = pd.DataFrame(list(stations.find()))
df = df[['call sign', 'location', 'frequency', 'county', 'usage', 'latitude', 'longitude']]
df.head(10)